In [1]:
import os
import sys
import numpy as np
import cv2

from data_loader import *
from fbs_config import TrainFBSConfig, InferenceFBSConfig
from fbs_dataset import FBSDataset

from mrcnn import model as modellib
from datahandler import DataHandler

from sklearn.metrics import f1_score
from scipy.ndimage import _ni_support
from scipy.ndimage.morphology import distance_transform_edt, binary_erosion,\
     generate_binary_structure
from tqdm import tqdm

from medpy.io import save
from math import ceil, floor

import skimage.color

ROOT_DIR = os.path.abspath('../../../')
sys.path.append(ROOT_DIR)

DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, 'logs')
DEFAULT_MODEL_DIR = os.path.join(DEFAULT_LOGS_DIR, 'mask_rcnn/kfold')

kernel = np.ones((5,5),np.uint8)

dh = DataHandler()

Using TensorFlow backend.


In [2]:
def destiny_directory(dice_score):
    pre = './data/eval/mask_rcnn/'
    if dice_score >= 98:
        return pre + 'dice_98_100/'
    elif dice_score >= 96:
        return pre + 'dice_96_98/'
    elif dice_score >= 94:
        return pre + 'dice_94_96/'
    elif dice_score >= 92:
        return pre + 'dice_92_94/'
    elif dice_score >= 90:
        return pre + 'dice_90_92/'
    elif dice_score >= 88:
        return pre + 'dice_88_90/'
    elif dice_score >= 85:
        return pre + 'dice_85_88'
    elif dice_score >= 80:
        return pre + 'dice_80_85/'
    elif dice_score >= 70:
        return pre + 'dice_70_80/'
    elif dice_score >= 60:
        return pre + 'dice_60_70/'
    else:
        return pre + 'dice_less_60'

def getFileName(fname):
    original_name = fname.split('/')[-1]
    original_name = original_name[:original_name.index('.')]
    return original_name

In [3]:
image_files, mask_files = load_data_files('data/kfold_data/')

skf = getKFolds(image_files, mask_files, n=10)

kfold_indices = []
for train_index, test_index in skf.split(image_files, mask_files):
    kfold_indices.append({'train': train_index, 'val': test_index})

In [4]:
def getDataset(val_index):
    image_val_files = np.take(image_files, val_index)
    mask_val_files = np.take(mask_files, val_index)

    val_files = ([image_val_files], [mask_val_files])

    dataset_val = FBSDataset()
    len_dataset_val = dataset_val.load_data(val_files)
    dataset_val.prepare()
    return dataset_val

In [5]:
def getDiceScore(ground_truth, prediction):
    #convert to boolean values and flatten
    ground_truth = np.asarray(ground_truth, dtype=np.bool).flatten()
    prediction = np.asarray(prediction, dtype=np.bool).flatten()    
    return f1_score(ground_truth, prediction)

In [6]:
 def hd(result, reference, voxelspacing=None, connectivity=1):
    hd1 = __surface_distances(result, reference, voxelspacing, connectivity).max()
    hd2 = __surface_distances(reference, result, voxelspacing, connectivity).max()
    hd = max(hd1, hd2)
    return hd

def hd95(result, reference, voxelspacing=None, connectivity=1):
    hd1 = __surface_distances(result, reference, voxelspacing, connectivity)
    hd2 = __surface_distances(reference, result, voxelspacing, connectivity)
    hd95 = np.percentile(np.hstack((hd1, hd2)), 95)
    return hd95

def __surface_distances(result, reference, voxelspacing=None, connectivity=1):
    result = np.atleast_1d(result.astype(np.bool))
    reference = np.atleast_1d(reference.astype(np.bool))
    if voxelspacing is not None:
        voxelspacing = _ni_support._normalize_sequence(voxelspacing, result.ndim)
        voxelspacing = np.asarray(voxelspacing, dtype=np.float64)
        if not voxelspacing.flags.contiguous:
            voxelspacing = voxelspacing.copy()

    footprint = generate_binary_structure(result.ndim, connectivity)

    if 0 == np.count_nonzero(result):
        raise RuntimeError('The first supplied array does not contain any binary object.')
    if 0 == np.count_nonzero(reference):
        raise RuntimeError('The second supplied array does not contain any binary object.')

    result_border = result ^ binary_erosion(result, structure=footprint, iterations=1)
    reference_border = reference ^ binary_erosion(reference, structure=footprint, iterations=1)

    dt = distance_transform_edt(~reference_border, sampling=voxelspacing)
    sds = dt[result_border]

    return sds

In [7]:
def evaluateMask(gt_mask, pred_mask):
    return getDiceScore(gt_mask, pred_mask), hd(gt_mask, pred_mask), hd95(gt_mask, pred_mask)
    

In [8]:
import random
def predictAll(inferenceFBSConfig, val_indices):
    model = modellib.MaskRCNN(mode='inference', config=inferenceFBSConfig, model_dir=DEFAULT_MODEL_DIR)
    inferenceFBSConfig.display()
    print(DEFAULT_MODEL_DIR)
    weights_path = model.find_last()
    print('Loading weights from %s'%weights_path)
    model.load_weights(weights_path, by_name=True)
    
    dice_scores = []
    hd_scores = []
    hd95_scores = []
    
    for image_index in tqdm(val_indices):
        #for saving
        fname = getFileName(image_files[image_index])
        not_used_full_image, hdr = dh.getImageData(image_files[image_index])
        dataset = getDataset(image_index)
                
        prediction = []
        gt_mask = []
    
        for img_id in dataset.image_ids:
            image, image_meta, class_ids, bbox, mask = modellib.load_image_gt(
            dataset, inferenceFBSConfig, img_id, use_mini_mask=False)
            
            results = model.detect([image], verbose=0)
            r = results[0]
            
            pred = r['masks']
            
            if(len(pred.shape) > 2 and pred.shape[2] == 0):
                pred = np.zeros((pred.shape[0],pred.shape[1],1))
            
            if(mask.shape[2] == 0):
                mask = np.zeros((pred.shape[0],pred.shape[1],1))
            
            pred = np.asarray(pred, dtype=np.uint8)
            pred = cv2.dilate(pred,kernel,iterations = 1)
                     
            prediction.append(pred)
            gt_mask.append(mask)
        
        pred_mask = np.asarray(prediction, dtype=np.bool)
        gt_mask = np.asarray(gt_mask, dtype=np.bool)
        
        dice_score, hd_score, hd95_score = evaluateMask(np.squeeze(gt_mask), pred_mask)
        
        dice_scores.append(dice_score)
        hd_scores.append(hd_score)
        hd95_scores.append(hd95_score)
        
        int_dice_score = floor(dice_score * 100)
        save_path = destiny_directory(int_dice_score)
        save_path = os.path.join(ROOT_DIR, save_path)
            
        save(pred, os.path.join(save_path, fname + '_mask_rcnn_' 
            + str(int_dice_score) + '.nii'), hdr)
    
    return dice_scores, hd_scores, hd95_scores

In [9]:
all_dice = []
all_hd = []
all_hd95 = []

for i in range(10):#len(kfold_indices)):    
    
    configParams = {'da': False,'tl': False, 'mask_dim': 28, 'wl': True, 'kfold_i': i}

    trainFBSConfig = TrainFBSConfig(**configParams)
    inferenceFBSConfig = InferenceFBSConfig(**configParams)    
    
    dice_scores, hd_scores, hd95_scores = predictAll(inferenceFBSConfig, kfold_indices[i]['val'])
    
    print('K%d results'%i)
    print('dice %f'%np.mean(dice_scores))
    print('hd %f'%np.mean(hd_scores))
    print('hd95 %f'%np.mean(hd95_scores))
    print()
    
    
    for dice_score, hd_score, hd95_score in zip(dice_scores, hd_scores, hd95_scores):
        all_dice.append(dice_score)
        all_hd.append(hd_score)
        all_hd95.append(hd95_score)


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        1
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            1
IMAGE_MAX_DIM                  256
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  256
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [256 256   1]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                  

  0%|          | 0/30 [00:00<?, ?it/s]

Re-starting from epoch 24



100%|██████████| 30/30 [03:21<00:00,  6.15s/it]11it/s]


K0 results
dice 0.859684
hd 54.266656
hd95 27.847512


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        1
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            1
IMAGE_MAX_DIM                  256
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  256
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [256 256   1]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POO

...

K8 results
dice 0.825090
hd 48.237304
hd95 28.699999


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        1
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            1
IMAGE_MAX_DIM                  256
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  256
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [256 256   1]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POO

  0%|          | 0/29 [00:00<?, ?it/s]

Re-starting from epoch 24



100%|██████████| 29/29 [03:20<00:00,  6.44s/it]03it/s]

K9 results
dice 0.827053
hd 51.325389
hd95 30.663025



In [10]:
print('dice')
for score in all_dice:
    print(score)

print()
print('hd')
for score in all_hd:
    print(score)

print()
print('hd95')
for score in all_hd95:
    print(score)
    

dice
0.8876387746426778
0.9147648425874839
0.8435244277168675
0.8056772307164097
0.7314435702714343
0.9457656103277424
0.9449687046775639
0.8918124552015504
0.9408690325640214
0.9212235556105406
0.9357778834634672
0.8960545638773205
0.8524872918492551
0.9467200549986529
0.9398688921095316
0.8466526512874107
0.9209163288880424
0.7858424434615705
0.5639129987306598
0.9303936384571101
0.1976731601731602
0.9261419516992678
0.9550800820434651
0.9409736103539484
0.9135369529885423
0.8548574277682739
0.9409967448911243
0.9375172961461179
0.830155343120519
0.8472649725289155
0.7757413983491821
0.8511676287919802
0.8594606287856937
0.8931742494302188
0.8883893398483523
0.8480427392838759
0.670226105246127
0.9027779608370348
0.16593677840592402
0.8398890385545775
0.8182631728381307
0.8566578920870553
0.8815625659649078
0.6000765186833119
0.8929038368489864
0.8788732394366198
0.8886230194292887
0.8661361291317291
0.9143365251770015
0.8829860401137609
0.8922105314492629
0.8785976153870965
0.860335